# Finding CHaracteristic function values for sample of transactions

## Key idea: For each combination of the item description, find the characteristic function

## Example: for subset {A,B}, the characteristic function value = sum over all invoices for price*quantity of subsets A, B, AB

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#reading the data
df = pd.read_excel('online_merged.xlsx', sheet_name='merged')

In [ ]:
df.head()

In [ ]:

df.describe()
df['Description'] = df['Description'].str.strip()

In [ ]:
df1 = df
df1.head()

In [ ]:
#finding price * quantity as total price per item per invoice
df1_price = df1[(df1[['Price','Quantity']] > 0).all(axis=1)]
df1.describe()

In [ ]:
# Checking for unique items

len(df.StockCode.unique())

In [ ]:
unq_desc = df.StockCode.unique()

In [ ]:
unq_desc

## Here, I have taken a sample of 5 distinct items and based on those items, i have samled the dataset. furthermore, from the sampled dataset, i have picked 50 random transactions to make the computation simpler. 

In [ ]:
#np.random.seed(123)
sample_desc = np.random.choice(unq_desc, size = 7)

sample_txn = df1[df1['StockCode'].isin(sample_desc)]

sample_txn = sample_txn.sample(n = 100)

len(sample_txn.Invoice.unique())

sample_txn.groupby(['StockCode']).count()

sample_txn['TotalPrice'] = sample_txn['Price']*sample_txn['Quantity']

sample_txn

basket = (sample_txn.groupby(['Invoice', 'Description'])['TotalPrice'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Invoice')) 
basket

In [ ]:
sample_txn.sort_values(['Invoice'])

In [ ]:
#finding average price of each item (for calculating shapley value)
avgs = sample_txn.groupby(['Description']).mean()
avgs

In [ ]:
# finding sum of quantities of each item (to calculate per item shapley value)
sums = sample_txn.groupby(['Description']).sum()
sums

In [ ]:
backupbasket = basket

In [ ]:
#adding few extra transactions for finding the combinations
basket.iloc[2]['FELT TOADSTOOL SMALL'] = 4.25
basket.iloc[5]['FELT TOADSTOOL SMALL'] = 4.25
basket.iloc[7]['SET OF 2 TEA TOWELS PING MICROWAVE'] = 11.80
basket.iloc[5]['DELUXE SEWING KIT'] = 11.90
basket.iloc[10]['IVORY CAFE HANGING LAMP'] = 31.2


In [ ]:

basket

In [ ]:
#Finding invoice wise sum

invoice_sum = list(basket.sum(axis = 1))


invoice_sum

In [ ]:
len(invoice_sum)

In [ ]:
basket.to_csv ('basket.csv', index = True, header=True)

In [ ]:
#finding distinct items to create subsets of items
list2 = list(basket.columns)

list2

In [ ]:
print('itemwise sum: ',basket.sum())
print('total sum: ', basket.sum().sum())

In [ ]:
# Creating subsets

import itertools
subsets = []
stuff = list2
for L in range(0, len(stuff)+1):
    for subset in itertools.combinations(stuff, L):
        subsets.append(list(subset))

subsets
    

In [ ]:
# finding the row wise items (have to optimise the step)

col_list_full = []
for index, row in basket.iterrows():
    col_list = []
    for col in basket:
            #if col in 
        if(basket.at[index,col]!=0.0):
            col_list.append(col)
    col_list_full.append(col_list)
    #print('-------------------------------------------------------------------------')
            #print(round(row.sum(),2))
col_list_full      

In [ ]:
# grouping the invoice and columns based on the subsets

before_grp = []
for i in range (len(basket)):
    temp = [str(col_list_full[i]),invoice_sum[i]]
    #print(col_list_full[i], ' ',invoice_sum[i])
    before_grp.append(temp)

before_grp

e_dataframe = pd.DataFrame(before_grp)
e_dataframe.columns = ['a', 'b']

e_dataframe

groupped_sum = pd.DataFrame(e_dataframe.groupby(['a']).sum())
groupped_sum.reset_index(level=0, inplace=True)
groupped_sum.columns = ['a', 'b']

In [ ]:
groupped_sum

In [ ]:
# finding characteristic function value for each group and assigning them to subset list
all_sums = []
for sub in subsets:
    sum1 = 0
    print('subset id:', subsets.index(sub))
    print("step1")
    #print(sub)
    print('------------')
    stuff = sub
    new_sub = []
    for L in range(1, len(stuff)+1):
        for subset in itertools.combinations(stuff, L):
            new_sub.append(list(subset))
    print(new_sub)
    for item in new_sub:
        index = groupped_sum[groupped_sum['a'] == str(item)].index
        if index.empty:
            print('None')
        else:
            print(item, index)
            sum1+=groupped_sum['b'][index].values[0]
    all_sums.append(sum1)
    print('------------------------------------------------------------')

In [ ]:
all_sums

In [ ]:
new_dict = {}
for i in range(len(all_sums)):
    new_dict.update({str(subsets[i]) : all_sums[i] })
    print(subsets[i], " ", all_sums[i])

In [ ]:
new_dict

In [ ]:
new_dict.get("['CHRISTMAS DECOUPAGE CANDLE']")

## Now we calculate shapley values based on the subset characteristic functions



### I haven't done it for these ones yet since the shapley value library got currupted on my system and i couldn't get time to do it by hand. 

In [ ]:
print(groupped_sum.a[1])
print(subsets[5])
str(groupped_sum.a[1]) == str(subsets[5])

In [ ]:
list2

In [ ]:
subsets

In [ ]:
#
#for i in range (len(subsets)):
#    sub = subsets[i]
#    inv = inverse_subsets[i]
#    for item in sub:
#        print(item)
#        for index, row in basket.iterrows():
#            if item in sub and item not in inv:
#                print(row[item])
#        print('----------------------------------------------------')

In [ ]:
subset_sums = []

for i in range len(subsets):
    
    sum1 = 0
    for item in subsets[i]:
        if item not in 
        sum1 = sum1 + basket[item].sum()
        #print(item, basket2[item].sum())
    subset_sums.append(sum1)
    
    

In [ ]:
for i in range(len(subsets)):
    print(subsets[i], " " , round(subset_sums[i],2) )
    print('---------------------------------------------------------------')

In [44]:
import shap

In [83]:
unq_desc = sorted(list(df.Description.unique()))
unq_desc

['10 COLOUR SPACEBOY PEN',
 '12 ASS ZINC CHRISTMAS DECORATIONS',
 '12 COLOURED PARTY BALLOONS',
 '12 DAISY PEGS IN WOOD BOX',
 '12 EGG HOUSE PAINTED WOOD',
 '12 HANGING EGGS HAND PAINTED',
 '12 IVORY ROSE PEG PLACE SETTINGS',
 '12 MESSAGE CARDS WITH ENVELOPES',
 '12 MINI TOADSTOOL PEGS',
 '12 PENCIL SMALL TUBE WOODLAND',
 '12 PENCILS SMALL TUBE POSY',
 '12 PENCILS SMALL TUBE RED RETROSPOT',
 '12 PENCILS SMALL TUBE RED SPOTTY',
 '12 PENCILS SMALL TUBE SKULL',
 '12 PENCILS TALL TUBE POSY',
 '12 PENCILS TALL TUBE RED RETROSPOT',
 '12 PENCILS TALL TUBE RED SPOTTY',
 '12 PENCILS TALL TUBE SKULLS',
 '12 PENCILS TALL TUBE WOODLAND',
 '12 PINK HEN+CHICKS IN BASKET',
 '12 PINK ROSE PEG PLACE SETTINGS',
 '12 RED ROSE PEG PLACE SETTINGS',
 '15 PINK FLUFFY CHICKS IN BOX',
 '15CM CHRISTMAS GLASS BALL 20 LIGHTS',
 '16 PC CUTLERY SET PANTRY DESIGN',
 '16 PIECE CUTLERY SET PANTRY DESIGN',
 '18PC WOODEN CUTLERY SET DISPOSABLE',
 '2 DAISIES HAIR COMB',
 '2 PICTURE BOOK EGGS EASTER BUNNY',
 '2 PICTURE BOOK EGGS EASTER CHICKS',
 '2 PICTURE BOOK EGGS EASTER DUCKS',
 '20 DOLLY PEGS RETROSPOT',
 '200 BENDY SKULL STRAWS',
 '200 RED + WHITE BENDY STRAWS',
 '24 HANGING EASTER EGGS FLORAL TUB',
 '3 BIRDS CANVAS SCREEN',
 '3 BLACK CATS W HEARTS BLANK CARD',
 '3 DRAWER ANTIQUE WHITE WOOD CABINET',
 '3 GARDENIA MORRIS BOXED CANDLES',
 '3 HEARTS HANGING DECORATION RUSTIC',
 '3 HOOK HANGER MAGIC GARDEN',
 '3 HOOK PHOTO SHELF ANTIQUE WHITE',
 '3 PIECE SPACEBOY COOKIE CUTTER SET',
 '3 PINK HEN+CHICKS IN BASKET',
 "3 RAFFIA RIBBONS 50'S CHRISTMAS",
 '3 RAFFIA RIBBONS VINTAGE CHRISTMAS',
 '3 ROSE MORRIS BOXED CANDLES',
 '3 STRIPEY MICE FELTCRAFT',
 '3 TIER CAKE TIN GREEN AND CREAM',
 '3 TIER CAKE TIN RED AND CREAM',
 '3 TIER SWEETHEART GARDEN SHELF',
 '3 TRADITIONAL COOKIE CUTTERS  SET',
 '3 WHITE CHOC MORRIS BOXED CANDLES',
 '3 WICK CHRISTMAS BRIAR CANDLE',
 '36 DOILIES DOLLY GIRL',
 '36 DOILIES SPACEBOY DESIGN',
 '36 DOILIES VINTAGE CHRISTMAS',
 '36 FOIL HEART CAKE CASES',
 '36 FOIL STAR CAKE CASES',
 '36 PENCILS TUBE POSY',
 '36 PENCILS TUBE RED RETROSPOT',
 '36 PENCILS TUBE RED SPOTTY',
 '36 PENCILS TUBE SKULLS',
 '36 PENCILS TUBE WOODLAND',
 '3D CHRISTMAS STAMPS STICKERS',
 '3D DOG PICTURE PLAYING CARDS',
 '3D HEARTS  HONEYCOMB PAPER GARLAND',
 '3D SHEET OF CAT STICKERS',
 '3D SHEET OF DOG STICKERS',
 '3D SHEET OF SEA WORLD STICKERS',
 '3D STICKERS CHRISTMAS STAMPS',
 '3D STICKERS TRADITIONAL CHRISTMAS',
 '3D STICKERS VINTAGE CHRISTMAS',
 '3D TRADITIONAL CHRISTMAS STICKERS',
 '3D VINTAGE CHRISTMAS STICKERS',
 '4 BLUE DINNER CANDLES SILVER FLOCK',
 '4 BURGUNDY WINE DINNER CANDLES',
 '4 FESTIVE GREEN DINNER CANDLES',
 '4 GOLD FLOCK CHRISTMAS BALLS',
 '4 IVORY DINNER CANDLES GOLD FLOCK',
 '4 IVORY DINNER CANDLES SILVER FLOCK',
 '4 LAVENDER BOTANICAL DINNER CANDLES',
 '4 LILY  BOTANICAL DINNER CANDLES',
 '4 PEAR BOTANICAL DINNER CANDLES',
 '4 PINK DINNER CANDLE SILVER FLOCK',
 '4 PINK FLOCK CHRISTMAS BALLS',
 '4 PURPLE FLOCK DINNER CANDLES',
 '4 ROSE PINK DINNER CANDLES',
 '4 SKY BLUE DINNER CANDLES',
 '4 TRADITIONAL SPINNING TOPS',
 '4 VANILLA BOTANICAL CANDLES',
 '4 WILDFLOWER BOTANICAL CANDLES',
 '5 HOOK BLACKBOARD ORGANISER',
 '5 HOOK HANGER MAGIC TOADSTOOL',
 '5 HOOK HANGER RED MAGIC TOADSTOOL',
 '5 STRAND GLASS NECKLACE AMBER',
 '5 STRAND GLASS NECKLACE AMETHYST',
 '5 STRAND GLASS NECKLACE BLACK',
 '5 STRAND GLASS NECKLACE CRYSTAL',
 "50'S CHRISTMAS GIFT BAG LARGE",
 "50'S CHRISTMAS PAPER GIFT BAG",
 '50CM METAL STRING WITH  7 CLIPS',
 '6 CHOCOLATE LOVE HEART T-LIGHTS',
 '6 CROCHET STRAWBERRIES',
 '6 EGG HOUSE PAINTED WOOD',
 "6 GIFT TAGS 50'S CHRISTMAS",
 '6 GIFT TAGS VINTAGE CHRISTMAS',
 '6 HOOK JEWELLERY STAND BLACK DRESS',
 '6 RABBIT CHAIN EASTER',
 '6 RIBBONS ELEGANT CHRISTMAS',
 '6 RIBBONS EMPIRE',
 '6 RIBBONS RUSTIC CHARM',
 '6 RIBBONS SHIMMERING PINKS',
 '6 ROCKET BALLOONS',
 '6 SEGMENT COLOURED SNACK BOWL',
 '60 CAKE CASES DOLLY GIRL DESIGN',
 '60 CAKE CASES VINTAGE CHRISTMAS',
 '60 GOLD AND SILVER FAIRY CAKE CASES',
 '60 TEATIME FAIRY CAKE CASES

In [80]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [84]:
for value in unq_desc:
    print(value)

10 COLOUR SPACEBOY PEN
12 ASS ZINC CHRISTMAS DECORATIONS
12 COLOURED PARTY BALLOONS
12 DAISY PEGS IN WOOD BOX
12 EGG HOUSE PAINTED WOOD
12 HANGING EGGS HAND PAINTED
12 IVORY ROSE PEG PLACE SETTINGS
12 MESSAGE CARDS WITH ENVELOPES
12 MINI TOADSTOOL PEGS
12 PENCIL SMALL TUBE WOODLAND
12 PENCILS SMALL TUBE POSY
12 PENCILS SMALL TUBE RED RETROSPOT
12 PENCILS SMALL TUBE RED SPOTTY
12 PENCILS SMALL TUBE SKULL
12 PENCILS TALL TUBE POSY
12 PENCILS TALL TUBE RED RETROSPOT
12 PENCILS TALL TUBE RED SPOTTY
12 PENCILS TALL TUBE SKULLS
12 PENCILS TALL TUBE WOODLAND
12 PINK HEN+CHICKS IN BASKET
12 PINK ROSE PEG PLACE SETTINGS
12 RED ROSE PEG PLACE SETTINGS
15 PINK FLUFFY CHICKS IN BOX
15CM CHRISTMAS GLASS BALL 20 LIGHTS
16 PC CUTLERY SET PANTRY DESIGN
16 PIECE CUTLERY SET PANTRY DESIGN
18PC WOODEN CUTLERY SET DISPOSABLE
2 DAISIES HAIR COMB
2 PICTURE BOOK EGGS EASTER BUNNY
2 PICTURE BOOK EGGS EASTER CHICKS
2 PICTURE BOOK EGGS EASTER DUCKS
20 DOLLY PEGS RETROSPOT
200 BENDY SKULL STRAWS
200 RED + WHITE 

FRENCH BLUE METAL DOOR SIGN, 0
FRENCH BLUE METAL DOOR SIGN, 1
FRENCH BLUE METAL DOOR SIGN, 2
FRENCH BLUE METAL DOOR SIGN, 3
FRENCH BLUE METAL DOOR SIGN, 4
FRENCH BLUE METAL DOOR SIGN, 5
FRENCH BLUE METAL DOOR SIGN, 6
FRENCH BLUE METAL DOOR SIGN, 7
FRENCH BLUE METAL DOOR SIGN, 8
FRENCH BLUE METAL DOOR SIGN, 9
FRENCH BOTTLE , LAVENDER
FRENCH BOTTLE, BRASSERIE DES ARTIST
FRENCH CARRIAGE LANTERN
FRENCH CHATEAU LARGE FRUIT BOWL
FRENCH CHATEAU LARGE PLATTER
FRENCH CHATEAU OVAL PLATTER
FRENCH CHATEAU SMALL FRUITBOWL
FRENCH EMROIDR LAVENDER SCENT HEART
FRENCH ENAMEL CANDLEHOLDER
FRENCH ENAMEL POT W LID
FRENCH ENAMEL SOAP DISH WITH LID
FRENCH ENAMEL UTENSIL HOLDER
FRENCH ENAMEL WATER BASIN
FRENCH FLORAL CUSHION COVER
FRENCH GARDEN SIGN BLUE METAL
FRENCH KITCHEN SIGN BLUE METAL
FRENCH LATTICE CUSHION COVER
FRENCH LAUNDRY SIGN BLUE METAL
FRENCH LAVENDER SCENT HEART
FRENCH PAISLEY CUSHION COVER
FRENCH STYLE EMBOSSED HEART CABINET
FRENCH STYLE STORAGE JAR BONBONS
FRENCH STYLE STORAGE JAR CAFE
FRENC

MIRRORED WALL ART LADIES
MIRRORED WALL ART PHOTO FRAMES
MIRRORED WALL ART POPPIES
MIRRORED WALL ART SKULLS
MIRRORED WALL ART SNOWFLAKES
MIRRORED WALL ART SPLODGES
MIRRORED WALL ART STARS
MIRRORED WALL ART TABLE LAMP
MISELTOE HEART WREATH
MISELTOE HEART WREATH CREAM
MISELTOE HEART WREATH WHITE
MISTLETOE HEART WREATH CREAM
MISTLETOE HEART WREATH GREEN
MIXED NUTS LIGHT GREEN BOWL
MOBILE VINTAGE HEARTS
MOCK LOBSTER FRIDGE MAGNET
MODERN FLORAL STATIONERY SET
MODERN VINTAGE COTTON SHOPPING BAG
MONEY BOX BISCUITS DESIGN
MONEY BOX CONFECTIONERY DESIGN
MONEY BOX FIRST ADE DESIGN
MONEY BOX FIRST AID DESIGN
MONEY BOX HOUSEKEEPING DESIGN
MONEY BOX KINGS CHOICE DESIGN
MONEY BOX POCKET MONEY DESIGN
MONSTERS STENCIL CRAFT
MONTANA DIAMOND CLUSTER EARRINGS
MONTANA DIAMOND CLUSTER NECKLACE
MOODY BOY  DOOR HANGER
MOODY GIRL DOOR HANGER
MOP PENDANT SHELL NECKLACE
MORE BUTTER METAL SIGN
MOROCCAN BEATEN METAL DISH
MOROCCAN BEATEN METAL DISH LARGE
MOROCCAN BEATEN METAL MIRROR
MOROCCAN TEA GLASS
MOSAIC CANDLE

RETRO PLASTIC 70'S TRAY
RETRO PLASTIC DAISY TRAY
RETRO PLASTIC ELEPHANT TRAY
RETRO PLASTIC POLKA TRAY
RETRO RED SPOTTY WASHING UP GLOVES
RETRO SISTERS EXPRESSO CUPS
RETRO SPORT PARTY BAG + STICKER SET
RETRO SPOT  CIGAR BOX MATCHES
RETRO SPOT CAKE STAND
RETRO SPOT CANDLE  LARGE
RETRO SPOT CANDLE  MEDIUM
RETRO SPOT CANDLE  SMALL
RETRO SPOT CERAMIC TOASTRACK
RETRO SPOT GIANT  TUBE MATCHES
RETRO SPOT LAMP
RETRO SPOT LARGE MILK JUG
RETRO SPOT MUG
RETRO SPOT PADDED SEAT CUSHION
RETRO SPOT PARTY BAG + STICKER SET
RETRO SPOT SMALL MILK JUG
RETRO SPOT SMALL TUBE MATCHES
RETRO SPOT STORAGE JAR
RETRO SPOT SUGAR JAM BOWL
RETRO SPOT TEA SET CERAMIC 11 PC
RETRO SPOT TRADITIONAL TEAPOT
RETRO SPOTS BUTTER DISH
RETRO SPOTS PUDDING BOWL
RETRO TIN ASHTRAY,REVOLUTIONARY
RETROSPOT BABUSHKA DOORSTOP
RETROSPOT CANDLE  LARGE
RETROSPOT CANDLE  MEDIUM
RETROSPOT CANDLE  SMALL
RETROSPOT CHILDRENS APRON
RETROSPOT CIGAR BOX MATCHES
RETROSPOT GIANT TUBE MATCHES
RETROSPOT HEART HOT WATER BOTTLE
RETROSPOT LAMP
RETROSP

WHITE BELL HONEYCOMB PAPER GARLAND
WHITE BIRD GARDEN DESIGN MUG
WHITE BITTY LIGHT CHAIN
WHITE BROCANTE SOAP DISH
WHITE CANDYSTUFT ARTIFICIAL FLOWER
WHITE CERAMIC PARTY SET (5PCS)
WHITE CHERRY LIGHTS
WHITE CHOCOLATE SCENT CANDLE
WHITE CHRISTMAS FLOCK DROPLET
WHITE CHRISTMAS GARLAND STARS TREES
WHITE CHRISTMAS HEART DECORATION
WHITE CHRISTMAS STAR DECORATION
WHITE CHRISTMAS TREE 60CM
WHITE CHRISTMAS TREE DECORATION
WHITE CHRYSANTHEMUMS ART FLOWER
WHITE DOVE HONEYCOMB PAPER GARLAND
WHITE ENAMEL FLOWER HAIR TIE
WHITE FOXGLOVE ARTIIFCIAL FLOWER
WHITE FRANGIPANI HAIR CLIP
WHITE FRANGIPANI NECKLACE
WHITE GLASS CHUNKY CHARM BRACELET
WHITE GLASS TV TRIPLE FRAME
WHITE GOOSE FEATHER CHRISTMAS TREE
WHITE GOOSE FEATHER TREE 60CM
WHITE GRAND BAROQUE PHOTO FRAME
WHITE HAND TOWEL WITH BUTTERFLY
WHITE HANGING BEADS CANDLE HOLDER
WHITE HANGING HEART T-LIGHT HOLDER
WHITE HEART CONFETTI IN TUBE
WHITE HEART OF GLASS BRACELET
WHITE HEARTS WIRE PLANT POT HOLDER
WHITE HONEYCOMB PAPER FAN
WHITE HONEYCOMB PAPER

In [85]:
unq_desc = unq_desc.split(' ')

AttributeError: 'list' object has no attribute 'split'

In [50]:
import numpy as np
import sklearn.cluster
import distance

words = unq_desc #Replace this line
words = np.asarray(words) #So that indexing with a list will work
lev_similarity = -1*np.array([[distance.jaccard(w1,w2) for w1 in words] for w2 in words])

affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping=0.5)
affprop.fit(lev_similarity)
for cluster_id in np.unique(affprop.labels_):
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

 - *RETRO SPOT MUG:* RED RETROSPOT MUG, RETRO SPOT MUG, SPOTS MUG
 - *RETRO SPOT SUGAR JAM BOWL:* BLUE SPOTS  WRAP, JUMBO BAG ALPHABET, JUMBO BAG APPLES, JUMBO BAG PEARS, JUMBO BAG RED RETROSPOT, JUMBO BAG STRAWBERRY, RED RETROSPOT JUMBO BAG , RED RETROSPOT SUGAR JAM BOWL, RETRO SPOT SUGAR JAM BOWL
 - *GIANT MEDINA STAMPED METAL BOWL :* AMBER DROP EARRINGS W LONG BEADS, BOMBS AWAY METAL SIGN, GIANT MEDINA STAMPED METAL BOWL , LARGE MEDINA STAMPED METAL BOWL , MEDIUM MEDINA STAMPED METAL BOWL , SET/4 BADGES WOODLAND ANIMAL, SMALL MEDINA STAMPED METAL BOWL , VINTAGE BEAD PINK JEWEL STAND
 - *STRAWBERRY CERAMIC TRINKET BOX:* BOX OF 6 MINI 50'S CRACKERS, BOX/12 MOTHER & CHICK IN BASKET/BOW, CERAMIC STRAWBERRY CAKE MONEY BANK, CERAMIC STRAWBERRY MONEY BOX, CERAMIC STRAWBERRY TRINKET TRAY, CORONA MEXICAN TRAY, OYSTER TRINKET BOX, PINK MINI STATIONERY SET W CASE, STRAWBERRY CERAMIC TRINKET BOX, STRAWBERRY CERAMIC TRINKET POT, SWEETHEART CERAMIC TRINKET BOX
 - *PINK CLEAR GLASS CANDLE PLATE:* 

In [86]:
len(unq_desc)


5083

In [87]:
list1 = ['CHOCOLATE BOX RIBBONS',
'DINOSAUR LUNCH BOX WITH CUTLERY',
'DINOSAUR LUNCHBOX WITH CUTLERY',
'EASTER BUNNY WITH BASKET ON BACK',
'EASTER BUNNY WREATH',
'ELEPHANT TOY WITH BLUE T-SHIRT',
'HEART BUTTONS JEWELLERY BOX',
'LUNCH BOX WITH CUTLERY FAIRY CAKES',
'LUNCH BOX WITH CUTLERY RETROSPOT',
'LUNCHBOX WITH CUTLERY FAIRY CAKES',
'LUNCHBOX WITH CUTLERY RETROSPOT',
'NEW BAROQUE IVORY CUSHION COVER',
'SKULL LUNCH BOX WITH CUTLERY',
'SKULL LUNCHBOX WITH CUTLERY',
'STRAWBERRY LUNCH BOX WITH CUTLERY',
'STRAWBERRY LUNCHBOX WITH CUTLERY',
'UTILTY CABINET WITH HOOKS']

In [88]:
#np.random.seed(123)
sample_desc = list1

sample_txn = df1[df1['Description'].isin(sample_desc)]

sample_txn = sample_txn.sample(n = 100)

len(sample_txn.Invoice.unique())

sample_txn.groupby(['StockCode']).count()

sample_txn['TotalPrice'] = sample_txn['Price']*sample_txn['Quantity']

sample_txn

basket = (sample_txn.groupby(['Invoice', 'Description'])['TotalPrice'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('Invoice')) 
basket

Description,CHOCOLATE BOX RIBBONS,DINOSAUR LUNCH BOX WITH CUTLERY,DINOSAUR LUNCHBOX WITH CUTLERY,ELEPHANT TOY WITH BLUE T-SHIRT,LUNCH BOX WITH CUTLERY FAIRY CAKES,LUNCH BOX WITH CUTLERY RETROSPOT,LUNCHBOX WITH CUTLERY FAIRY CAKES,SKULL LUNCH BOX WITH CUTLERY,STRAWBERRY LUNCH BOX WITH CUTLERY,STRAWBERRY LUNCHBOX WITH CUTLERY
Invoice,,,,,,,,,,
493965,0.0,0.00,0.0,3.75,0.0,0.0,0.0,0.0,0.00,0.0
494148,0.0,0.00,0.0,0.00,0.0,0.0,126.0,0.0,0.00,0.0
496564,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,15.3
497909,0.0,0.00,0.0,3.75,0.0,0.0,0.0,0.0,0.00,0.0
499010,0.0,0.00,5.1,0.00,0.0,0.0,0.0,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...
572653,2.5,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
572887,0.0,0.00,0.0,0.00,0.0,15.3,0.0,0.0,0.00,0.0
575607,0.0,4.96,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0
